# Running Tile2Net on Colab

If you don't have access to a physical GPU on your computer, or your GPU is not powerful enough, you can use the following workflow to use the tool!

There are two ways to use Tile2Net - via the command line or via imported python modules. Using a Colab workflow, the first way is a bit more straightforward, but both are possible.

You will notice exclamation points `!` and percent symbols `%` at the front of many code blocks in this notebook. Both are Colab syntax used to execute bash commands and "magic" commands in the colab environment. The relevant magic commands will be explained as they come up.

## Command Line Interface

### Step 0: Mount Google Drive

This step is optional, and is only necessary if you plan to upload your own tiles to the tool.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Step 1: Clone the Repository

In [1]:
!git clone https://github.com/VIDA-NYU/tile2net.git

Cloning into 'tile2net'...
remote: Enumerating objects: 5561, done.
remote: Counting objects: 100% (1116/1116), done.
remote: Compressing objects: 100% (316/316), done.
remote: Total 5561 (delta 939), reused 800 (delta 800), pack-reused 4445 (from 3)
Receiving objects: 100% (5561/5561), 4.58 MiB | 10.45 MiB/s, done.
Resolving deltas: 100% (3911/3911), done.


Once the repository has cloned, you can use the magic `%cd` command to go into the resulting    tile2net `directory`. It is important to use the magic command rather than calling `!cd`, because the latter creates a temporary subshell in which the command is called, whereas the former will persist the directory change.

In [2]:
!cd tile2net
!pwd

/content


In [3]:
%cd tile2net
!pwd

/content/tile2net
/content/tile2net


Always make sure you're up to date with the latest version of the code!

In [4]:
!git pull

Already up to date.


### Step 2: Activate a Virtual Environment and Install Dependencies

When using Colab, it is not necessary to create a conda environment, because each session creates its own environment. The following command installs all of the dependencies present in the [requirements-dev.txt](https://github.com/VIDA-NYU/tile2net/blob/main/requirements-dev.txt) file (That's what the period at the end does!)

In [9]:
import os

repo_path = '/content/tile2net'
pyproject_path = os.path.join(repo_path, 'pyproject.toml')

if os.path.exists(pyproject_path):
    with open(pyproject_path, 'r') as f:
        lines = f.readlines()

    new_lines = []
    modified = False
    for line in lines:
        # Corrected: search for 'requires-python' instead of 'python_requires'
        if line.strip().startswith('requires-python'):
            # Modify the line to be more permissive, including Python 3.12
            new_lines.append('requires-python = ">=3.10, <3.13"\n')
            modified = True
        else:
            new_lines.append(line)

    if modified:
        with open(pyproject_path, 'w') as f:
            f.writelines(new_lines)
        print(f"Modified {pyproject_path} to allow Python 3.12.")
    else:
        print(f"Could not find 'requires-python' in {pyproject_path} or it was already permissive (should not happen with the given error).")
else:
    print(f"Error: {pyproject_path} not found. Please ensure the tile2net repository is cloned.")

# Original installation command
# In Colab, the following two lines are unnecessary, but they are very important when working locally
# !conda create --name testenv python==3.11
# !conda activate testenv
!python -m pip install -e .

Modified /content/tile2net/pyproject.toml to allow Python 3.12.
Obtaining file:///content/tile2net
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Using cached centerline-1.1.1-py2.py3-none-any.whl.metadata (4.1 kB)
  Using cached jupyter-1.1.1-py2.py3-none-any.whl.metadata (2.0 kB)
  Using cached osmnx-2.0.7-py3-none-any.whl.metadata (4.9 kB)
  Using cached shapely-2.0.1.tar.gz (275 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached argh-0.29.4-py3-none-any.whl.metadata (9.3 kB)
  Using cached coolname-2.2.0-py2.py3-none-any.whl.metadata (6.2 kB)
  Using cached runx-0.0.11-py3-none-any.whl.metadata (18 kB)
  Using cached myst_parser-4.0.1-py3-none-any.whl.metadata (5.5 kB)
  Using cached sphinx_rtd_theme-3.0.2-py2.py3-none-a

### Step 3: Start Generating a Network!

You're pretty much done! All you need to do is call the same bash commands from the quick start command line tutorial, but with a `!` in front.

You can call `!python -m tile2net generate --help` or `!python -m tile2net inference --help` for more details on the generate and inference functions, respectively.

For example:

In [10]:
!python -m tile2net generate -l "42.35555189953313, -71.07168915322092, 42.35364837213307, -71.06437423368418" -n example -o "content/output"  | python -m tile2net inference

INFO       NumExpr defaulting to 2 threads.
INFO       NumExpr defaulting to 2 threads.
INFO       Using Massachusetts as the source at location=(42.3536483721, -71.0716891532, 42.3555518995, -71.0643742337)
INFO       Using base_tilesize=256 from source
INFO       Stitching 12 tiles...
INFO       96 tiles missing out of 96 total.
INFO       All 96 tiles are on disk.
           Stitching 6 tiles...                   : 100% 6/6 [00:00<00:00, 11.64it/s]
INFO       Dumping to content/output/example/tiles/example_256_info.json
INFO       Inferencing. Segmentation results will not be saved.
INFO       Downloading weights for segmentation, this may take a while...
INFO       Weights downloaded.
INFO       Using a single GPU.
INFO       Using Per Image based weighted loss
INFO       Using Cross Entropy Loss
INFO       Loading weights from: checkpoint=/content/tile2net/src/tile2net/raster/resources/assets/weights/satellite_2021.pth
INFO       init weights from normal distribution
INFO       lo

### Step 4: Explore and Save Your Results

Now that you've generated your stitched tiles and created your pedestrian network, it's time to explore!

Given that your output directory is `output_dir`, you will see the following folders:

- `output_dir/tiles/static`: the raw, unstitched orthorectified tiles used during segmentation
- `output_dir/tiles/stitched`: the stitched tiles
- `output_dir/segmentation/*`: metadata about the segmentation process used
- `output_dir/polygons/*`: shapefiles of polygons created from the segmentation results
- `output_dir/network/*`: the final pedestrian network shapefiles, ready to be visualized in any GIS software!

In Colab, you can download these files using the `!zip` command:

In [12]:
%cd content/output
!zip -r example.zip example

from google.colab import files
files.download('example.zip')

/content/tile2net/content/output
  adding: example/ (stored 0%)
  adding: example/tiles/ (stored 0%)
  adding: example/tiles/static/ (stored 0%)
  adding: example/tiles/static/ma/ (stored 0%)
  adding: example/tiles/static/ma/256_19/ (stored 0%)
  adding: example/tiles/static/ma/256_19/158641_193929.jpg (deflated 2%)
  adding: example/tiles/static/ma/256_19/158645_193927.jpg (deflated 2%)
  adding: example/tiles/static/ma/256_19/158642_193930.jpg (deflated 2%)
  adding: example/tiles/static/ma/256_19/158639_193931.jpg (deflated 1%)
  adding: example/tiles/static/ma/256_19/158646_193925.jpg (deflated 2%)
  adding: example/tiles/static/ma/256_19/158647_193931.jpg (deflated 2%)
  adding: example/tiles/static/ma/256_19/158643_193932.jpg (deflated 1%)
  adding: example/tiles/static/ma/256_19/158641_193925.jpg (deflated 1%)
  adding: example/tiles/static/ma/256_19/158648_193926.jpg (deflated 2%)
  adding: example/tiles/static/ma/256_19/158647_193930.jpg (deflated 2%)
  adding: example/tiles/

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Python Modules



To use Google Colab with the python module version of tile2net, the process is almost identical to the process described in the quick start module tutorial, with one caveat: you must restart your runtime after installing your libraries, but before running any commands that may require those libraries

First, follow steps 0-2 as directed in the [Command Line Interface](#command-line-interface) section.

**Then, go to `Runtime>Restart Session` or use the shortcut `CTRL+M .`**

After this, you can use the tile2net module as the Quick Start Module tutorial describes

In [1]:
import tile2net

location = "41.010361394056375, -74.14833643482527, 40.949516438466226, -74.06819594124879"
raster = tile2net.Raster(
    location=location,
    # name='boston common',
    output_dir='./example_dir',
    zoom=19
)
raster.Project

INFO       NumExpr defaulting to 2 threads.
INFO       Geocoding (40.979938916261304, -74.10826618803702), this may take a while...
INFO       Geocoded '(40.979938916261304, -74.10826618803702)' to
	'76, North Irving Street, Ridgewood, Bergen County, New Jersey, 07450, United States'
INFO       Using NewJersey as the source at location=(40.9495164385, -74.1483364348, 41.0103613941, -74.0681959412)
INFO       Using base_tilesize=256 from source


tile2net.raster.project.Project